In [1]:
import os
import re
import pandas as pd
import io
import base64
from openai import OpenAI
from collections import defaultdict
import pdfplumber # 이미지 추출을 위해 유지
import filetype
import camelot # 테이블 추출을 위해 추가
from unstructured.partition.pdf import partition_pdf

from pprint import pprint

In [14]:
# --- 1. AI 클라이언트 및 유틸리티 함수 정의 (기존과 동일) ---
client = OpenAI() # API 키는 환경 변수에 설정 권장

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def generate_image_description(image_path):
    print(f"  - 이미지 설명 생성 요청: {os.path.basename(image_path)}")
    base64_image = encode_image_to_base64(image_path)
    image_ext = os.path.splitext(image_path)[1].lstrip('.')
    
    system_prompt = """
    당신은 이미지를 분석하여 검색 시스템(RAG)을 위한 메타데이터를 생성하는 AI 전문가입니다.
    주어진 이미지의 모든 시각적, 텍스트적 요소를 추출하여 구조화된 설명을 제공하는 임무를 맡았습니다.
    """

    user_prompt = f"""
    이 이미지를 RAG 시스템에서 효과적으로 검색할 수 있도록, 아래 [분석 지침]에 따라 한글로 상세히 설명해주세요.

    [분석 지침]
    1.  **종합 요약 (1~2문장)**: 이미지의 핵심 주제와 내용을 간결하게 요약해주세요.

    2.  **주요 구성요소 및 객체**: 이미지에 포함된 중요한 사물, 인물, 아이콘, 그래프 요소 등을 구체적으로 나열해주세요. (예: 막대그래프, 돋보기 아이콘, 노트북을 사용하는 사람 등)

    3.  **이미지 내 텍스트 (OCR)**: 이미지에 보이는 모든 텍스트를 그대로 옮겨 적어주세요. 텍스트가 없다면 '텍스트 없음'이라고 명시해주세요.

    4.  **시각적 특징 및 스타일**: 이미지의 전체적인 색상 톤, 구도, 스타일(예: 사진, 일러스트, 스크린샷, 다이어그램), 분위기 등을 설명해주세요.

    5.  **핵심 키워드 (쉼표로 구분)**: 검색에 사용될 만한 핵심 키워드를 5개 이상 나열해주세요.

    ---
    위 지침에 따라 분석 결과를 생성해주세요.
    """
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": [
                {"type": "text", "text": user_prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/{image_ext};base64,{base64_image}"}}
            ]}
        ],
        max_tokens=500  # 다양한 정보를 담기 위해 토큰 수를 늘리는 것을 권장합니다.
    )
    description = response.choices[0].message.content.strip()
    print(f"  - 설명 생성 완료: {description[:30]}...")
    return description

def generate_table_summary(table_csv_str):
    if len(table_csv_str) > 4000:
        table_csv_str = table_csv_str[:4000]
        
    system_prompt = """
        당신은 최고의 데이터 분석가입니다. 당신의 임무는 CSV 형식의 데이터를 구조적으로 분석하고, 비전문가도 이해하기 쉽게 핵심 내용을 요약하는 것입니다.
        항상 다음 분석 절차를 엄격히 준수하여 답변을 생성해 주세요.
        """
        
    user_prompt = f"""
        다음 CSV 데이터를 보고 아래 [분석 지침]에 따라 상세한 분석 보고서를 작성해 주세요.

        --- 데이터 시작 ---
        {table_csv_str}
        --- 데이터 끝 ---

        [분석 지침]
        1.  **표의 주제**: 이 표가 무엇에 대한 데이터인지 한 문장으로 명확하게 정의하세요.
        2.  **구조 설명**:
            - 각 행(row)이 무엇을 나타내는지 설명하세요. (예: 각 행은 하나의 결제 단계, 사용자 정보 등을 나타냅니다.)
            - 각 열(column)의 이름과 그것이 어떤 정보를 담고 있는지 설명하세요.
        3.  **핵심 정보 및 수치**:
            - 표에서 가장 중요한 핵심 정보를 3~5가지 항목으로 요약하세요.
            - 특히, 다른 항목과 구별되는 **구체적인 수치, 비율(%), 조건, 기간** 등이 있다면 반드시 포함하여 설명하세요.
        4.  **패턴 또는 특이사항 (선택 사항)**:
            - 데이터를 전체적으로 보았을 때 발견할 수 있는 패턴, 경향성 또는 특이한 점이 있다면 언급하세요.
        5. 데이터는 행 또는 열들의 결합으로 되어있을 수도 있습니다. 행과 열 모두 신중히 보세요
        """
    
    print("  - 테이블 요약 생성 요청...")
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=800  # 상세한 설명을 위해 토큰 수를 조금 늘리는 것을 권장합니다.
    )
    summary = response.choices[0].message.content.strip()
    print(f"  - 요약 생성 완료: {summary[:30]}...")
    return summary

# --- 2. AI 기능이 통합된 처리 함수 (camelot 데이터 형식에 맞게 수정) ---

def process_table_with_ai_camelot(table_df, output_dir, page_num, table_index):
    """camelot으로 추출한 테이블 DataFrame을 처리하고 AI 요약을 생성합니다."""
    try:
        df = table_df
        table_filename = f"p{page_num}_tbl{table_index}.csv"
        table_path = os.path.join(output_dir, "tables", table_filename)
        df.to_csv(table_path, index=False, encoding="utf-8-sig")
        
        # AI 요약을 위해 DataFrame을 CSV 문자열로 변환
        csv_string = df.to_csv(index=False)
        description = generate_table_summary(csv_string)
        
        return {"path": table_path, "page": page_num, "dataframe": df, "description": description}
    except Exception as e:
        print(f"  - 테이블 처리 중 오류 발생: {e}")
        return None

# process_image_with_ai 함수는 기존과 동일하게 pdfplumber를 사용하므로 변경 없음
def process_image_with_ai(image_data, output_dir, page_num, image_index):
    try:
        image_bytes = image_data['stream'].get_data()
        kind = filetype.guess(image_bytes)
        image_ext = kind.extension if kind else "png"
        image_filename = f"p{page_num}_img{image_index}.{image_ext}"
        image_path = os.path.join(output_dir, "images", image_filename)
        with open(image_path, "wb") as img_file:
            img_file.write(image_bytes)
        description = generate_image_description(image_path)
        return {"path": image_path, "page": page_num, "description": description}
    except Exception as e:
        print(f"  - 이미지 처리 중 오류 발생: {e}")
        return None

# --- 3. camelot을 사용하도록 수정한 메인 함수 ---

def create_integrated_markdown_from_camelot(pdf_path):
    output_dir = os.path.basename(pdf_path).split(".")[0]
    os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "tables"), exist_ok=True)

    # 1단계: pdfplumber로 이미지, camelot으로 테이블 미리 추출
    print("1단계: 미디어 정보 미리 추출 중 (이미지: pdfplumber, 테이블: camelot)...")
    
    # pdfplumber로 이미지 추출
    images_by_page = defaultdict(list)
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            images_by_page[page.page_number].extend(page.images)
    print(f"총 {sum(len(v) for v in images_by_page.values())}개의 이미지 정보를 찾았습니다.")

    # camelot으로 테이블 추출
    # flavor='lattice'는 선이 있는 표에 적합, 선이 없는 표는 'stream' 사용
    tables_by_page = defaultdict(list)
    try:
        tables = camelot.read_pdf(pdf_path, pages='all', flavor='lattice', suppress_stdout=True)
        for table in tables:
            tables_by_page[table.page].append(table.df)
        print(f"총 {sum(len(v) for v in tables_by_page.values())}개의 테이블 정보를 찾았습니다.")
    except Exception as e:
        print(f"Camelot으로 테이블 추출 중 오류 발생: {e}")

    # 2단계: unstructured로 PDF 요소 순서대로 파티셔닝 (기존과 동일)
    print("2단계: unstructured로 문서 구조 분석 중 (hi_res 전략 사용)...")
    elements = partition_pdf(filename=pdf_path, infer_table_structure=True, strategy="hi_res")

    # 3단계: 요소 순회하며 최종 마크다운 및 데이터 생성
    print("3단계: 텍스트, 테이블, 이미지를 통합하고 AI로 내용을 보강합니다...")
    final_markdown_parts = []
    extracted_tables = []
    extracted_images = []
    page_counters = defaultdict(lambda: {'tables': 0, 'images': 0})

    for el in elements:
        page_num = el.metadata.page_number
        
        if "unstructured.documents.elements.Table" in str(type(el)):
            table_index = page_counters[page_num]['tables']
            if table_index < len(tables_by_page[page_num]):
                # camelot이 추출한 DataFrame을 처리
                raw_table_df = tables_by_page[page_num][table_index]
                table_data = process_table_with_ai_camelot(raw_table_df, output_dir, page_num, table_index)
                if table_data:
                    placeholder = f"\n\n[[-- TABLE: Page {page_num}, Index {table_index} | Path: {table_data['path']} --]]\n**표 요약:** {table_data['description']}\n\n"
                    final_markdown_parts.append(placeholder)
                    extracted_tables.append(table_data)
                    page_counters[page_num]['tables'] += 1
            
        elif "unstructured.documents.elements.Image" in str(type(el)):
            image_index = page_counters[page_num]['images']
            if image_index < len(images_by_page[page_num]):
                # pdfplumber가 추출한 이미지 처리
                raw_image_data = images_by_page[page_num][image_index]
                image_data = process_image_with_ai(raw_image_data, output_dir, page_num, image_index)
                if image_data:
                    relative_path = os.path.relpath(image_data['path'], output_dir).replace(os.sep, '/')
                    md_link = f"\n\n![{image_data['description']}]({relative_path})\n\n"
                    final_markdown_parts.append(md_link)
                    extracted_images.append(image_data)
                    page_counters[page_num]['images'] += 1
        
        else:
            final_markdown_parts.append(el.text)

    # 4단계: 최종 결과 정리 및 저장 (기존과 동일)
    print("4단계: 최종 결과 정리 및 저장...")
    final_markdown = "\n\n".join(final_markdown_parts)
    output_filename = os.path.join(output_dir, "integrated_markdown_camelot.md")
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write(final_markdown)
    
    print("\n--- 작업 완료 ---")
    return {
        "output_dir": output_dir,
        "integrated_markdown_file": output_filename,
        "images": extracted_images,
        "tables": extracted_tables,
        "integrated_markdown_content": final_markdown,
    }

# --- 코드 실행 예제 ---
pdf_path = "datasets/manual.pdf" # 실제 PDF 파일 경로로 변경해주세요.
extracted_data = create_integrated_markdown_from_camelot(pdf_path)

# # 결과 확인
# print(f"\nAI 요약/설명이 포함된 통합 마크다운 파일이 '{extracted_data['integrated_markdown_file']}'에 저장되었습니다.")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


1단계: 미디어 정보 미리 추출 중 (이미지: pdfplumber, 테이블: camelot)...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


총 0개의 이미지 정보를 찾았습니다.


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


총 2개의 테이블 정보를 찾았습니다.
2단계: unstructured로 문서 구조 분석 중 (hi_res 전략 사용)...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


3단계: 텍스트, 테이블, 이미지를 통합하고 AI로 내용을 보강합니다...
  - 테이블 요약 생성 요청...
  - 요약 생성 완료: ## 분석 보고서

### 1. 표의 주제
이 표는 협...
  - 테이블 요약 생성 요청...
  - 요약 생성 완료: ### 분석 보고서

1. **표의 주제**: 이 표는...
4단계: 최종 결과 정리 및 저장...

--- 작업 완료 ---


In [21]:
import torch
import re
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document  # Document 클래스를 import 합니다.

# --- 1. 메타데이터를 포함하여 청킹하고 Document 객체를 생성하는 함수 ---

def create_documents_from_markdown(markdown_content, source_path):
    """
    논리적 청킹을 수행하고, 각 청크를 메타데이터가 포함된 LangChain Document 객체로 변환합니다.

    Args:
        markdown_content (str): 청킹할 마크다운 전체 텍스트
        source_path (str): 원본 파일 경로 (메타데이터에 사용)

    Returns:
        list[Document]: 내용과 메타데이터가 포함된 Document 객체 리스트
    """
    # 기존의 논리적 청킹 함수를 그대로 사용합니다.
    text_chunks = chunk_markdown_logically(markdown_content)
    
    documents = []
    for chunk_text in text_chunks:
        metadata = {"source": source_path}  # 모든 청크에 기본 출처 정보 추가
        
        # 청크 내용을 기반으로 추가 메타데이터 파싱
        if chunk_text.startswith("[[-- TABLE:"):
            metadata['type'] = 'table'
            # 정규표현식을 사용하여 페이지 번호와 경로 추출
            page_match = re.search(r"Page (\d+)", chunk_text)
            path_match = re.search(r"Path: (.*?)\s*--]]", chunk_text)
            if page_match:
                metadata['page'] = int(page_match.group(1))
            if path_match:
                metadata['table_path'] = path_match.group(1).strip()

        elif chunk_text.startswith("!["):
            metadata['type'] = 'image'
            # 정규표현식을 사용하여 페이지 번호와 이미지 설명 추출
            page_match = re.search(r"Image from page (\d+)", chunk_text)
            desc_match = re.search(r"!\[(.*?)\]\(", chunk_text)
            if page_match:
                metadata['page'] = int(page_match.group(1))
            if desc_match:
                metadata['description'] = desc_match.group(1).strip()
        else:
            metadata['type'] = 'text'

        # Document 객체 생성
        doc = Document(page_content=chunk_text, metadata=metadata)
        documents.append(doc)
        
    return documents

# --- 2. 기존 로직 실행 및 all_chunks 생성 ---

# chunk_markdown_logically 함수는 제공된 코드에 이미 정의되어 있다고 가정합니다.
def chunk_markdown_logically(markdown_content):
    """
    문서의 구조(제목, 표, 이미지)를 이해하여 논리적인 단위로 청킹합니다.
    RAG 시스템에 가장 효과적인 방법입니다.
    """
    # 1. 기본 블록으로 분리
    blocks = markdown_content.split('\n\n')
    
    chunks = []
    current_chunk = ""

    for block in blocks:
        block = block.strip()
        if not block:
            continue

        # 2. 규칙에 따라 청크 생성
        is_table_placeholder = block.startswith("[[-- TABLE:")
        is_image_link = block.startswith("![")
        # 제목으로 사용될 수 있는 패턴들 (필요에 따라 추가)
        is_heading = block.startswith(('□', '○', '※')) or re.match(r'^[0-9]+\s|^\w+\s', block) and len(block) < 50

        if is_table_placeholder or is_image_link:
            # 테이블/이미지는 독립적인 청크로 처리
            if current_chunk:
                chunks.append(current_chunk.strip())
            chunks.append(block)
            current_chunk = ""
        elif is_heading:
            # 제목은 다음 블록과 합치기 위해, 진행 중인 청크를 먼저 저장
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = block
        else:
            # 일반 텍스트는 현재 청크(제목 또는 이전 텍스트)에 추가
            if current_chunk:
                current_chunk += "\n\n" + block
            else:
                current_chunk = block
    
    # 마지막 남은 청크 추가
    if current_chunk:
        chunks.append(current_chunk.strip())
        
    return chunks

source_file_path = "manual/integrated_markdown_camelot.md"

with open(source_file_path, "r", encoding="utf-8") as f:
    content = f.read()

# 'all_chunks' 변수는 이제 단순 문자열 리스트가 아닌,
# 내용과 메타데이터를 모두 포함한 Document 객체의 리스트가 됩니다.
all_chunks = create_documents_from_markdown(content, source_file_path)

pprint(all_chunks)

[Document(metadata={'source': 'manual/integrated_markdown_camelot.md', 'type': 'text'}, page_content='Ⅴ 사업비 집행 및 정산지침\n\nOE'),
 Document(metadata={'source': 'manual/integrated_markdown_camelot.md', 'type': 'text'}, page_content='사업비 지급 및 집행기간'),
 Document(metadata={'source': 'manual/integrated_markdown_camelot.md', 'type': 'text'}, page_content='□ 사업비 지급'),
 Document(metadata={'source': 'manual/integrated_markdown_camelot.md', 'type': 'table', 'page': 1, 'table_path': 'manual\\tables\\p1_tbl0.csv'}, page_content='[[-- TABLE: Page 1, Index 0 | Path: manual\\tables\\p1_tbl0.csv --]]\n**표 요약:** ## 분석 보고서'),
 Document(metadata={'source': 'manual/integrated_markdown_camelot.md', 'type': 'text'}, page_content='### 1. 표의 주제\n이 표는 협약 체결 및 지급 비율에 따른 신청 시기별 데이터를 나타내고 있습니다.\n\n### 2. 구조 설명\n- **행(row)**:\n  - 각 행은 신청 시기와 그에 따른 지급 비율을 나타내고 있습니다. \n  - 첫 번째 행(헤더)은 각 회차를 구분하는 마킹을 포함하고 있습니다(1회차, 2회차 등).\n  \n- **열(column)**:\n  - 각 열의 이름은 신청 시기와 그에 대한 지급 비율로 나뉘어 있습니다.\n  - 1~2분기와 3~4분기에 각각 2개의 신청 시기와

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

source_file_path = "manual/integrated_markdown_camelot.md"

with open(source_file_path, "r", encoding="utf-8") as f:
    file_content = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512, # 청크 크기를 1000으로 설정
    chunk_overlap=100,  # 중첩 크기를 150으로 설정
    length_function=len, # 텍스트 길이를 계산할 함수 지정
)

# .split_text() 메서드를 사용하여 텍스트를 분할합니다.
split_texts = text_splitter.split_text(file_content)
pprint(split_texts)

['Ⅴ 사업비 집행 및 정산지침\n'
 '\n'
 'OE\n'
 '\n'
 '사업비 지급 및 집행기간\n'
 '\n'
 '□ 사업비 지급\n'
 '\n'
 '\n'
 '\n'
 '[[-- TABLE: Page 1, Index 0 | Path: manual\\tables\\p1_tbl0.csv --]]\n'
 '**표 요약:** ## 분석 보고서\n'
 '\n'
 '### 1. 표의 주제\n'
 '이 표는 협약 체결 및 지급 비율에 따른 신청 시기별 데이터를 나타내고 있습니다.\n'
 '\n'
 '### 2. 구조 설명\n'
 '- **행(row)**:\n'
 '  - 각 행은 신청 시기와 그에 따른 지급 비율을 나타내고 있습니다. \n'
 '  - 첫 번째 행(헤더)은 각 회차를 구분하는 마킹을 포함하고 있습니다(1회차, 2회차 등).\n'
 '  \n'
 '- **열(column)**:\n'
 '  - 각 열의 이름은 신청 시기와 그에 대한 지급 비율로 나뉘어 있습니다.\n'
 '  - 1~2분기와 3~4분기에 각각 2개의 신청 시기와 지급 비율이 나열되어 있습니다.\n'
 '  - 예를 들어, "협약 체결 시"라는 열은 해당 시점에 20%의 지급 비율을 나타냅니다.',
 '### 3. 핵심 정보 및 수치\n'
 '1. **신청 시기: 협약 체결 시** - 지급 비율 20%\n'
 '2. **신청 시기: 협약 체결 후 2개월 이내** - 지급 비율 46%\n'
 '3. **신청 시기: 중간 평가 후 1개월 이내** - 지급 비율 9%\n'
 '4. **신청 시기: 4분기(10월)** - 지급 비율 25%\n'
 '\n'
 '### 4. 패턴 또는 특이사항\n'
 '- 협약 체결 후 2개월 이내에 지급 비율이 가장 높은 46%로 나타나, 지원이 신속하게 이어질 가능성이 있음을 보여줍니다.\n'
 '- 중간 평가 후 1개월 이내에는 지급 비율이 9%로 가장 낮아, 이 단계에서의 지급이 상대적으로 적다는것은 평가 과정이 지급에 큰 '
 '영향을 미친다는 것을 의미

In [23]:
# --- 3. Vector Store 생성 및 저장 ---

print("\n--- Vector Store 생성 시작 ---")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"임베딩 모델을 위한 디바이스 설정: {device}")

embedding_model = HuggingFaceEmbeddings(
    model_name="dragonkue/BGE-m3-ko",
    model_kwargs={'device': device},
    encode_kwargs={'normalize_embeddings': True},
)

# Chroma.from_documents 함수는 Document 객체 리스트를 받도록 설계되어 있습니다.
vectorstore = Chroma.from_documents(
    documents=all_chunks,
    embedding=embedding_model,
    persist_directory="./chroma_db"  # DB를 저장할 디렉토리
)

print("\n--- Vector Store 생성 및 저장 완료 ---")
print(f"'{vectorstore._persist_directory}' 디렉토리에 {len(all_chunks)}개의 청크가 저장되었습니다.")


--- Vector Store 생성 시작 ---
임베딩 모델을 위한 디바이스 설정: cuda

--- Vector Store 생성 및 저장 완료 ---
'./chroma_db' 디렉토리에 39개의 청크가 저장되었습니다.


In [34]:
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

retriever = vectorstore.as_retriever()


prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If the user asks for a simple answer, summarize the key points.
If the question is unrelated to the context in the regulations, respond with "관련 정보를 찾을 수 없습니다."
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)


# llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
llm = OllamaLLM(model="gemma3:4b")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [37]:
question = "2회차는 협약 채결 후 몇개월 이내야?"
# 테이블 정보를 최대한 정밀하게 추출할 수 있도록 하기
answer = chain.invoke(question)
print(answer)

협약 체결 후 2개월 이내에 46%의 지급 비율을 가집니다.


<class 'langchain_core.documents.base.Document'>
